# Дата-экспедиция - поиск информации о жертвователях партии "Единая Россия" #

In [1]:
import requests
import pandas as pd
import numpy as np
import json
from time import gmtime, strftime

data_file = 'data.xlsx'# файл, в который были собраны данные из сканов финансовых отчетов

## Очистка данных ##

In [2]:
data = pd.read_excel(data_file)

# удалим столбец, в котором суммы представлены строками
del data['money'] 

# переименуем столбец, в котором суммы являются числами
data = data.rename(columns = {'money_cleared' : 'money'}) 

# уберем нечитаемые символы из названий регионов
data.region = [r.strip() for r in data.region.tolist()] 

# если год где-то записан в виде целых чисел, запишем в виде строк
data.year =[str(y) for y in data.year.tolist()] 

# если ИНН где-то записан в виде целых чисел, запишем в виде строк
data.inn = [str(i) for i in data.inn.tolist()] 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719 entries, 0 to 1718
Data columns (total 6 columns):
year         1719 non-null object
company      1719 non-null object
inn          1719 non-null object
money        1719 non-null float64
region_id    1719 non-null int64
region       1719 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 80.7+ KB


In [3]:
len(data.inn.unique()) # сколько уникальных ИНН в наборе

1104

In [4]:
data.money.mean() # средний размер пожертвования за все годы

7196806.511419429

In [5]:
data.money = data.money / 1000000 # дальше всем суммы в миллионах рублей

analysis = pd.DataFrame.from_dict({'number_of_cases' : data.groupby('year').size().tolist(), 
                                   'money_sum' : data.groupby('year')['money'].sum().tolist(),
                                   'money_mean' : data.groupby('year')['money'].mean().tolist(),
                                   'year' : sorted(list(data.year.unique()))})

analysis

,money_mean,money_sum,number_of_cases,year
0,5.849650,1017.839181,174,2006
1,0.356000,1.780000,5,2007
2,5.415053,1018.029980,188,2008
3,17.123870,2191.855370,128,2009
4,9.191440,1351.141662,147,2010
5,10.833092,2816.603895,260,2011
6,7.593926,1002.398199,132,2012
7,5.693645,666.156416,117,2013
8,6.862861,1324.532263,193,2014
9,2.615929,980.973427,375,2015


In [6]:
# сохраним файл для дальнейшей визуализации
analysis.to_csv('analysis.csv')

Сформируем .csv файл и преобразуем данные в формат json. В последнем случае ключами будут жертвователи - к каждому жертвователю припишем его ИНН, регион, год и сумму пожертвования

In [7]:
data.to_csv('party_donations.csv')

In [8]:
def create_single_company(inn): # эта функция создает json для одной компании и возвращает его
        df = data[data.inn == inn]
        d = {'inn' : inn,
    # у компании названия могут писаться по-разному, в том числе могла быть смена имени, поэтому запишем все возможные
             'company_names' : list(set(df.company)), 
             'region' : min(set(df.region)),
    # donations - словарь с ключами-годами и значениями - суммами пожертвования в этот год
             'donations' : dict(zip(df.year.tolist(), df.money.tolist()))}
        return d
    
def create_json(data): # эта функция создает json всех компаний и записывает его в файл

    all_inns = data.inn.unique()
    all_companies = []
  
    for inn in all_inns:
        all_companies.append(create_single_company(inn))
        
    with open('party_donations_json.json', 'w') as outfile:
        json.dump(all_companies, outfile)
        
create_json(data)

** Посмотрим на структуру json для одной компании **

In [9]:
create_single_company('7743064487')

{'company_names': ['Региональный общественный фонд поддержки Всероссийской политической партии "Единая Россия"',
  'Региональный общественный фонд поддержки ВП "Единство и Отечество"-Единая Россия',
  'Региональный общественный фонд поддержки Всероссийской политической партии "Единая Россия "'],
 'donations': {'2006': 6.8739999999999997,
  '2008': 5.474837,
  '2009': 32.642000000000003,
  '2010': 26.830492,
  '2011': 34.288915000000003,
  '2012': 40.700000000000003,
  '2013': 32.0,
  '2014': 38.314571999999998},
 'inn': '7743064487',
 'region': 'г. Москва'}

** Создадим файлы с метаданными **

In [10]:
# метаданные для .json файла

meta_data = {'description' : 'В текущем наборе данных представлена информация о пожертвованиях юридических лиц партии "Единая Россия", начиная с 2006 года. Источник данных - финансовые отчеты партии, опубликованные ЦИК РФ.',
             'versiondate' : strftime("%Y-%m-%d", gmtime()), 
             'columns': [{'name': 'company_names',
                         'caption': 'Названия компании-жертвователя',
                         'type': 'list',
                         'subcolumns_type': 'string'},
                        
                        {'name': 'inn',
                         'caption': 'ИНН компании-жертвователя',
                         'type': 'string',
                         'subcolumns_type': np.nan},
                        
                        {'name': 'region',
                         'caption': 'Регион регистрации компании-жертвователя',
                         'type': 'string',
                         'subcolumns_type': np.nan},
                        
                        {'name': 'donations',
                         'caption': 'Суммы пожертвований компании',
                         'type': 'dict',
                         'subcolumns_type': {'dict_keys' : 'year', 
                                             'dict_keys_caption' : 'Год пожертвования', 
                                             'dict_keys_type' : 'string', 
                                             'dict_values' : 'sums', 
                                             'dict_values_caption' : 'Сумма пожертвования (в год)', 
                                             'dict_values_type' : 'float'}
                        }]
            }

        
with open('party_donations_json_meta.json', 'w') as outfile:
    json.dump(meta_data, outfile)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1719 entries, 0 to 1718
Data columns (total 6 columns):
year         1719 non-null object
company      1719 non-null object
inn          1719 non-null object
money        1719 non-null float64
region_id    1719 non-null int64
region       1719 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 80.7+ KB


In [12]:
# метаданные для .csv файла

def create_csv_metadata(df):
    # забираем названия колонок
    column_names = list(df.columns)
    
    # забираем типы данных из колонок (при условии, что, как и положено, в колонке данные одного типа)
    types = []
    replace = ["'", '64', 'numpy.', '>']
    for col in list(df.columns):
        type_ = str(type(df[col].tolist()[0])).split()[1]
        for char in replace:
            type_ = type_.replace(char, '')
        types.append(type_)
        
    return pd.DataFrame.from_dict({'field_name' : column_names, 'data_type' : types})

csv_meta = create_csv_metadata(data)

description = ['Год, когда было совершено пожертвование', 
               'Названия компании-жертвователя', 
               'ИНН компании-жертвователя',
              'Суммы пожертвований компании', 
               'Идентификатор региона регистрации компании-жертвователя',
               'Регион регистрации компании-жертвователя']

csv_meta['description'] = description
csv_meta

,data_type,field_name,description
0,str,year,"Год, когда было совершено пожертвование"
1,str,company,Названия компании-жертвователя
2,str,inn,ИНН компании-жертвователя
3,float,money,Суммы пожертвований компании
4,int,region_id,Идентификатор региона регистрации компании-жер...
5,str,region,Регион регистрации компании-жертвователя


In [13]:
csv_meta.to_csv('party_donations_csv_meta.csv')

## Поиск госконтрактов, заключенных с компаниями, жертвовавшими партии Единая Россия. ##

In [14]:
# Используем API Clearspending
URL_CONTRACTS = 'http://openapi.clearspending.ru/restapi/v3/contracts/select/?'

CUST_INN = 'customerinn=' 
SUP_INN = 'supplierinn='

inns = [str(inn) for inn in data.inn.unique()]# список ИНН уникальных жертвователей партии

In [15]:
# список url для обращения к API в каждом году

url_list_2010 = [(URL_CONTRACTS + SUP_INN + inn + '&daterange=01.01.2010-31.12.2010') for inn in inns]
url_list_2011 = [(URL_CONTRACTS + SUP_INN + inn + '&daterange=01.01.2011-31.12.2011') for inn in inns]
url_list_2012 = [(URL_CONTRACTS + SUP_INN + inn + '&daterange=01.01.2012-31.12.2012') for inn in inns]
url_list_2013 = [(URL_CONTRACTS + SUP_INN + inn + '&daterange=01.01.2013-31.12.2013') for inn in inns]
url_list_2014 = [(URL_CONTRACTS + SUP_INN + inn + '&daterange=01.01.2014-31.12.2014') for inn in inns]
url_list_2015 = [(URL_CONTRACTS + SUP_INN + inn + '&daterange=01.01.2015-31.12.2015') for inn in inns]
url_list_2016 = [(URL_CONTRACTS + SUP_INN + inn + '&daterange=01.01.2016-31.12.2016') for inn in inns]
url_list_2017 = [(URL_CONTRACTS + SUP_INN + inn + '&daterange=01.01.2017-31.12.2017') for inn in inns]

** Функция `total_contracts_as_supplier` ищет, сколько контрактов компании получили в определенном году, возвращает в виде словаря **

In [16]:
def total_contracts_as_supplier(url_list):
    total_contracts = {}
    
    for url in url_list:
        page = requests.get(url)
     
        if page.status_code != 404:
            contracts = page.json()['contracts']
            
            # приписали число контрактов, заключенных компанией-поставщиком
            total_contracts[url[73:83]] = contracts['total'] 
        else:
            total_contracts[url[73:83]] = 0       

        
    return total_contracts

** Запишем все контракты с 2012 года в таблицу `all_contracts`. **

In [ ]:
all_contracts = {'2012' : total_contracts_as_supplier(url_list_2012), '2013' : total_contracts_as_supplier(url_list_2013), 
                 '2014' : total_contracts_as_supplier(url_list_2014), '2015' : total_contracts_as_supplier(url_list_2015), 
                 '2016' : total_contracts_as_supplier(url_list_2016), '2017' : total_contracts_as_supplier(url_list_2017)}

all_contracts_df = pd.DataFrame.from_dict(all_contracts)

# Запишем в файл

all_contracts_df.to_csv('all_contracts_df.csv')